Experiment with OpenCV for finding the pool table edges within an image.

This version assumes a full-sized vertical image of the table. It is looking
for the marker pattern along the side of the table.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math
import copy

In [ ]:
def ImgShow(images, resolution=120):
    plt.rcParams['figure.dpi'] = resolution
    
    cnt = len(images)
    if cnt > 20:
        # Top-level object is an image, make it an array of 1
        images = [images]
        cnt = len(images)
        
    cols = cnt
    fig, ax = plt.subplots(1, cnt)    
    if 1 == cnt:
        ax = [ax]
    for i in range(0, len(images)):
        dims = len(images[i].shape)
        if 3 == dims:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_BGR2RGB)
        else:
            imgDisp = cv.cvtColor(images[i], cv.COLOR_GRAY2RGB)
            
        ax[i].axis('off')
        ax[i].imshow(imgDisp)
        
#Handy color switcher
clrs = []
clrs.append((255,0,0))
clrs.append((0,255,0))
clrs.append((0,0,255))
clrs.append((255,255,0))
clrs.append((255,0,255))
clrs.append((0,255,255))
iClr = 0

def Color():
    global iClr
    iClr += 1
    return clrs[iClr % len(clrs)]
    
clr = Color()

In [ ]:
# Load the test image
imageOriginal = cv.imread("../test/images/long_all.jpg")

In [ ]:
#Find the edges using flood, start with a blur
blur = int(7)
imageBlur = cv.GaussianBlur(imageOriginal, (blur,blur), cv.BORDER_DEFAULT)

# Edge detect using canny
if False:
    imageCanny = cv.Canny(imageBlur, 100, 175)
else:
    imageCannyMarksOnly = cv.imread("../test/images/CannyMarksOnly.jpg")
    imageCanny = cv.Canny(imageCannyMarksOnly, 100, 175)

ImgShow([imageBlur, imageCanny],220)

In [ ]:
# Contours
contours, b, = cv.findContours(imageCanny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
imgDrawnContours = imageOriginal.copy()
cv.drawContours(imgDrawnContours, contours, -1, (0,255,0),3)
ImgShow([imageCanny,imgDrawnContours])

In [ ]:
# Find circles

class Circle:
    def __init__(self, x,y,r):
        self.x = x
        self.y = y
        self.r = r
        
    def center(self):
        return (self.x, self.y)
    
    def point(self):
        return (int(self.x+0.5), int(self.y+0.5))
    
    def radius(self):
        return self.r
    
circles = []
for points in contours:
    (x,y),radius = cv.minEnclosingCircle(points)
    circles.append(Circle(x,y,radius))
    
#sort by radius
circles.sort(key=lambda s: s.radius()) # sorts using lambda function

# print out the data
imgCircles = imageOriginal.copy()
clrs = []
clrs.append((255,0,0))
clrs.append((0,255,0))
clrs.append((0,0,255))
iClr = 0
for circle in circles:
    #print("{0}\t{1}\t{2}".format(circle.radius(), circle.center()[0], circle.center()[1]))
    center_coordinates = (int(circle.center()[0]+0.5), int(circle.center()[1]+0.5))
    cv.circle(imgCircles, center_coordinates, int(circle.radius()),clrs[iClr % 3])	
    iClr = iClr+1
    
    
ImgShow([imgCircles], 200)
#cv.imwrite("../test/images/dotz.jpg", imgCircles)

In [ ]:
# Handy line class
class LineP:
    def __init__(self, circleA, circleB):
        x1 = circleA.center()[0]
        y1 = circleA.center()[1]
        x2 = circleB.center()[0]
        y2 = circleB.center()[1]
        rise = y2 - y1
        run = x2 - x1
        if run != 0.0:
            self.m = rise/run
        else:
            self.m = 10E10
        self.b = y1 - self.m * x1
        self.radius = (circleA.radius() + circleB.radius()) / 2.0
        self.circles = []
        self.circles.append(circleA)
        self.circles.append(circleB)
        
    def val(self):
        return self.m, self.b
        
    def angleRad(self):
        return np.arctan(self.m)
    
    def angleDeg(self):
        return self.angleRad() * 180.0 / math.pi
    
    def pointCount(self):
        return len(self.circles)
    
    def circleRadius(self):
        return self.radius
    
    def addCircle(self, circle):
        self.circles.append(circle)
        
    def distance(self, point):
        # This doesn't work right
        a = self.m
        b = -1
        c = self.b        
        x = point[0]
        y = point[1]
        d = abs(a*x + b*y + c) / math.sqrt(a*a + b*b)
        return d
    
    def endpoints(self):
        # Scan for the farthest points
        d2Max = 0.0
        for i in range(0, len(self.circles)-1):
            for j in range(i+1, len(self.circles)):
                dx = self.circles[i].center()[0] - self.circles[j].center()[0]
                dy = self.circles[i].center()[1] - self.circles[j].center()[1]
                d2 = dx*dx + dy*dy
                if d2 > d2Max:
                    d2Max = d2
                    p1 = self.circles[i].point()
                    p2 = self.circles[j].point()
        return p1,p2

# Test the endpoints
c1 = Circle(0.0,0.0,1.0)
c2 = Circle(10.0,0.0,1.0)
c3 = Circle(20.0,0.0,1.0)
line = LineP(c1, c2)
line.addCircle(c1)
line.addCircle(c3)
line.addCircle(c2)
pt1, pt2 = line.endpoints()
print(pt1, pt2)
m,b = line.val()
print(m,b)

In [ ]:
# Find the lines that the dots line up on
        
#Between every two points, there is a line. Find all of the possible lines.
linesAll = []
imgLinesAll = imageOriginal.copy()
maxRadiusDiff = 6.0
minCircleRadius = 5.0
for i in range(0, len(circles)-1):
    for j in range(i+1, len(circles)):
        circleA = circles[i]
        circleB = circles[j]
        
        # Skip circles that are too small
        if circleB.radius() < minCircleRadius or circleA.radius() < minCircleRadius:
            continue
        
        # Don't create lines between circles of different sizes
        radiusDiff = np.abs(circleB.radius() - circleA.radius())
        if(radiusDiff > maxRadiusDiff):
            continue   
        linesAll.append(LineP(circleA, circleB))
        cv.line(imgLinesAll, circleA.point(), circleB.point(), Color(), 3)

print("All Lines ", len(linesAll))
ImgShow([imgLinesAll],150)    


In [ ]:
# Extract vertical lines

imgLinesVertical = imageOriginal.copy()
angleLimitV = 20.0 # Only accept lines +/- this angle
linesVertical = []
for line in linesAll:        
    angle = line.angleDeg()
    #print(angle)
    if(angle > 90-angleLimitV and angle < 90+angleLimitV or angle > -90-angleLimitV and angle < -90+angleLimitV):
        linesVertical.append(line)
        p1,p2 = line.endpoints()
        cv.line(imgLinesVertical, p1,p2, Color(), 3)

print("Vertical Lines ", len(linesVertical))

ImgShow([imgLinesVertical],150)    


In [ ]:
# Add each circle to all lines that it lies upon
linesVertical2 = copy.deepcopy(linesVertical)
for circle in circles:
    # Find the line this might fit on. Give it to all lines that match.
    for line in line:
        if line.distance(circle.center()) > maxRadiusDiff:
            continue
        radDiff = abs(circle.radius() - line.circleRadius())
        line.addCircle(circle)
        if(radDiff <= maxRadiusDiff):
            line.addCircle(circle)  # Close to the line and withing radius tolerance

In [ ]:
# Extract only the lines that have enough circles that land on it
imgLinesPruned = imageOriginal.copy()
linesPruned= []
# Grab the lines with more than 3 points
for line in linesAll2:
    if line.pointCount() >= 6:
        linesPruned.append(line)
        p1,p2 = line.endpoints()
        cv.line(imgLinesPruned, p1,p2, Color(), 4)
        
print("Pruned Lines len=",len(linesPruned))
        
ImgShow([imgLinesPruned], 150)
for line in linesPruned:
    m,b = line.val()
    #print("line: ", m, b)

In [ ]:
# Find the leftmost and rightmost lines
rows = imageOriginal.shape[0]
cols = imageOriginal.shape[1]
ymid = rows / 2.0
minLeft = cols
maxRight = 0
for line in linesPruned:
    m,b = line.val()
    xmid = (ymid-b)/m
    
    # Left-most?
    if(xmid < minLeft):
        minLeft = xmid
        lineLeft = line
    if(xmid > maxRight):
        maxRight = xmid
        lineRight = line
        
imgLinesSides = imageOriginal.copy()
p1,p2 = lineLeft.endpoints()
cv.line(imgLinesSides, p1,p2, Color(), 4)
p1,p2 = lineRight.endpoints()
cv.line(imgLinesSides, p1,p2, Color(), 4)
ImgShow([imgLinesSides], 150)